In [1]:
import pandas as pd

# Use case reale: `MiMocko` 🛵

Il business vuole provare a capire quali viaggi sono più a rischio di risultare in un incidente.

Carichiamo i datasets di interesse: per ora limitiamoci a quello dei viaggi e degli incidenti

In [2]:
path = '../../../data'

viaggi = pd.read_csv(f'{path}/viaggi.csv', sep='*', decimal=',')
incidenti = pd.read_csv(f'{path}/incidenti.csv')

## Data preprocessing

#### 👉 Molte di queste elaborazioni le abbiamo già viste nelle lezioni precedenti, quindi andremo veloci

### Viaggi

In [3]:
viaggi.head(2)

,idUtente,idVeicolo,caricaBatteriaInizio,dueCaschiDisponibili,luogoRitiro,timestampRitiro,luogoConsegna,timestampConsegna,distanzaPercorsa,tempoViaggio,punteggioViaggio,caricaBatteriaFine
0,u5954,v33,100.0%,YES,"[(44.394923, 8.943928), (44.394857, 8.943474),...","['2018-10-01 01:08:57', '2018-10-01 01:09:02',...","[(44.433961, 8.95907), (44.434021, 8.958851), ...","['2018-10-01 01:21:57', '2018-10-01 01:22:02',...",4.5046 km,0:13:20,7.55,90.2%
1,u1478,v33,91.8%,NaN,"[(44.433997, 8.958993), (44.433596, 8.959153),...","['2018-10-01 02:09:21', '2018-10-01 02:09:26',...","[(44.396783, 8.940881), (44.396705, 8.940446),...","['2018-10-01 02:21:06', '2018-10-01 02:21:11',...",4.3622 km,0:12:05,7.78,81.8%


Controlliamo se ci sono valori mancanti

In [4]:
pd.isnull(viaggi).sum(axis=0)

idUtente                    0
idVeicolo                   0
caricaBatteriaInizio        0
dueCaschiDisponibili    37308
luogoRitiro                 0
timestampRitiro             0
luogoConsegna               0
timestampConsegna           0
distanzaPercorsa            0
tempoViaggio             6350
punteggioViaggio         1203
caricaBatteriaFine          0
dtype: int64

Vogliamo quindi:
    
* trasformare in `float` le colonne `caricaBatteriaInizio`, `distanzaPercorsa`, `tempoViaggio`, `caricaBatteriaFine`
* trasformare in `int` le colonna `dueCaschiDisponibili`
* estrarre un unico valore per ciascuna delle colonne `luogoRitiro`, `timestampRitiro`, `luogoConsegna`, `timestampConsegna`

gestendo i valori mancanti delle colonne `dueCaschiDisponibili`, `tempoViaggio`, `punteggioViaggio`

#### Valori mancanti

⚠️ scegliamo di eliminare i sample con dati mancanti, ma potremmo rischiare di perdere _più_ eventi di un tipo (incidente/no) rispetto all'altro, introducendo _selection bias_... ⚠️

**N.B.**: un'alternativa è sfruttare l'imputer di sklearn a valle dello split tra train e test.

_Approfondimento (BONUS):_
* [imputation](https://scikit-learn.org/stable/modules/impute.html)
* [data leakage during preprocessing](https://scikit-learn.org/stable/common_pitfalls.html#data-leakage-during-pre-processing)

In [5]:
viaggi.dropna(inplace=True)

In [6]:
pd.isnull(viaggi).sum(axis=0)

idUtente                0
idVeicolo               0
caricaBatteriaInizio    0
dueCaschiDisponibili    0
luogoRitiro             0
timestampRitiro         0
luogoConsegna           0
timestampConsegna       0
distanzaPercorsa        0
tempoViaggio            0
punteggioViaggio        0
caricaBatteriaFine      0
dtype: int64

#### Conversione a `float`
Per comodità, creiamo nuove colonne con il dato "ripulito"

In [7]:
viaggi['carica_batteria_inizio'] = viaggi['caricaBatteriaInizio'].apply(lambda x: float(x.replace('%', ''))/100)
viaggi['carica_batteria_fine'] = viaggi['caricaBatteriaFine'].apply(lambda x: float(x.replace('%', ''))/100)
viaggi['distanza_percorsa'] = viaggi['distanzaPercorsa'].apply(lambda x: float(x.replace(' km', '')))
viaggi['tempo_viaggio'] = viaggi['tempoViaggio'].apply(lambda x: pd.to_timedelta(x).total_seconds())

viaggi[['carica_batteria_inizio', 'carica_batteria_fine', 'distanza_percorsa', 'tempo_viaggio']].dtypes

carica_batteria_inizio    float64
carica_batteria_fine      float64
distanza_percorsa         float64
tempo_viaggio             float64
dtype: object

#### Conversione ad `int`


In [8]:
set(viaggi['dueCaschiDisponibili'])

{'NO', 'YES'}

In [9]:
viaggi['due_caschi_disponibili'] = viaggi['dueCaschiDisponibili'].map({'YES': 1, 'NO': 0})

In [10]:
set(viaggi['due_caschi_disponibili'])

{0, 1}

#### Manipolazione luogo e timestamp di ritiro e consegna 
Estraiamo la data e l'ora di consegna e ritiro in due colonne separate, a partire dal timestamp preprocessato

Scegliamo l'ultimo campionamento disponibile dell'evento _consegna_: essendo interessati agli incidenti, è verosimile che il campionamento si sia interrotto in corrispondenza dell'incidente.

<font color='orange'><b><i>L'esecuzione della prossima cella impiega qualche secondo</b></i></font>

In [11]:
# selezioniamo ultimo campionamento del timestamp di consegna
viaggi['timestamp'] = viaggi['timestampConsegna'].apply(lambda x: pd.to_datetime(x[1:-1].split(',')[-1]))

# estraiamo data, anno, mese, giorno, ora e minuto
viaggi['data'] = viaggi['timestamp'].apply(lambda x: x.date())
viaggi['anno'] = viaggi['data'].apply(lambda x: x.year)
viaggi['mese'] = viaggi['data'].apply(lambda x: x.month)
viaggi['giorno'] = viaggi['data'].apply(lambda x: x.day)
viaggi['tempo'] = viaggi['timestamp'].apply(lambda x: str(x.time())[:5])
viaggi['ora'] = viaggi['tempo'].apply(lambda x: int(x[:2]))
viaggi['minuto'] = viaggi['tempo'].apply(lambda x: int(x[3:]))

# estraiamo coordinate lat lon dell'ultimo campionamento GPS sia di ritiro che consegna
viaggi['luogo_ritiro'] = viaggi['luogoRitiro'].apply(lambda x: tuple(map(lambda y: float(y), x.split(' (')[-1][:-2].split(', '))))
viaggi['luogo_consegna'] = viaggi['luogoConsegna'].apply(lambda x: tuple(map(lambda y: float(y), x.split(' (')[-1][:-2].split(', '))))
viaggi['lat_ritiro'] = viaggi['luogo_ritiro'].apply(lambda x: x[0])
viaggi['lon_ritiro'] = viaggi['luogo_ritiro'].apply(lambda x: x[1])
viaggi['lat_consegna'] = viaggi['luogo_consegna'].apply(lambda x: x[0])
viaggi['lon_consegna'] = viaggi['luogo_consegna'].apply(lambda x: x[1])

# facciamo pulizia delle colonne ausiliarie
viaggi.drop(['timestamp', 'tempo', 'luogo_ritiro', 'luogo_consegna'], axis=1, inplace=True)

Selezioniamo ora solo le colonne preprocessate 

In [12]:
viaggi = viaggi[[
    'idUtente', 'idVeicolo', 'carica_batteria_inizio', 
    'carica_batteria_fine', 'distanza_percorsa', 'tempo_viaggio',
    'due_caschi_disponibili', 'anno', 'mese', 'giorno', 'ora',
    'minuto', 'lat_ritiro', 'lon_ritiro', 'lat_consegna', 
    'lon_consegna', 'punteggioViaggio'
    ]]

In [13]:
pd.isnull(viaggi).sum(axis=0)

idUtente                  0
idVeicolo                 0
carica_batteria_inizio    0
carica_batteria_fine      0
distanza_percorsa         0
tempo_viaggio             0
due_caschi_disponibili    0
anno                      0
mese                      0
giorno                    0
ora                       0
minuto                    0
lat_ritiro                0
lon_ritiro                0
lat_consegna              0
lon_consegna              0
punteggioViaggio          0
dtype: int64

### Incidenti

In [14]:
incidenti.head(2)

,providerEvento,targaVeicolo,costoEvento,dataEvento,oraEvento
0,Sonia Fantoni,X8V2VM,"178,00 €","Jun, 05, 2020",forse 18 e 45
1,Morena Fibonacci,X8T2WT,"196,00 €","Sep, 24, 2019",forse 18:52


Vogliamo:
    
* ricavare idUtente ed idVeicolo a partire risp. da nome utente e targa veicolo (per poter incrociare con dataset viaggi)
* uniformare il formato di data evento e ora evento rispetto a quanto contenuto nel dataset di viaggi

Serve quindi coinvolgere anche i restanti due dataset: utenti e scooter (per ora prendiamo da loro solo lo stretto necessario).

#### Recupero `idUtente` e `idVeicolo`

In [15]:
utenti = pd.read_csv(f'{path}/utenti.csv', sep='|')
scooter = pd.read_csv(f'{path}/scooter.csv', sep=';')

In [16]:
utenti.head(2)

,ID UTENTE,NOME,COGNOME,SESSO,DATA di NASCITA,LUOGO di NASCITA,C.F.,INDIRIZZO E-MAIL,INDIRIZZO,LAT INDIRIZZO,LON INDIRIZZO,PATENTE #,DATA ULTIMO RINNOVO PATENTE,CONSENSO al TRATTAMENTO dei DATI,TITOLO di STUDIO
0,u6586,Dario,Nibali,M,12/09/1986,Torino,NBLDRA86P12L219E,NaN,"GENOVA, VIA VAL D'ASTICO, 156","44,4946921","8,908447",U19713653J,2019-03-30,False,NaN
1,u2188,Giacinto,Ferraris,M,24/05/1986,Carrara,FRRGNT86E24B832A,ferraris86@vodafone.it,"GENOVA, VIA SAMBUGO, 10","44,4491083","8,6963344",U18283370F,2019-08-19,False,diploma di scuola secondaria superiore (4-5 anni)


In [17]:
scooter.head(2)

,Veicolo,Activation Number,Versione n°,Targa,Costo,Data Registrazione
0,v114,16,3,X8X2NL,"9.860,00 €",01/20/2020
1,v89,43,2,X8VG3N,"5.860,00 €",06/01/2019


Per legare idUtente a nome e cognome operiamo in due step:
    
* creiamo una colonna che contenga una stringa dove concateniamo nome e cognome, es. da `['Mario', 'Rossi']` a `'Mario Rossi'`
* creiamo un dizionario che mappi ciascun nome e cognome sul corrispondente idUtente

In [18]:
utenti['nome_cognome'] = utenti['NOME'] + ' ' + utenti['COGNOME']
nomi2id = utenti.set_index('nome_cognome')['ID UTENTE'].to_dict()

Ripetiamo quanto sopra, in maniera analoga, per legare targa veicolo all'id

In [19]:
targhe2id = scooter.set_index('Targa')['Veicolo'].to_dict()

Aggiungiamo le colonne con gli id al dataset incidenti

In [20]:
incidenti['idUtente'] = incidenti['providerEvento'].map(nomi2id)
incidenti['idVeicolo'] = incidenti['targaVeicolo'].map(targhe2id)

#### Processing `oraEvento` e `dataEvento`

In [21]:
incidenti['data'] = incidenti['dataEvento'].apply(lambda x: pd.to_datetime(x).date())
incidenti['anno'] = incidenti['data'].apply(lambda x: x.year)
incidenti['mese'] = incidenti['data'].apply(lambda x: x.month)
incidenti['giorno'] = incidenti['data'].apply(lambda x: x.day)

Per l'ora evento, sfruttiamo un contenuto bonus della lezione sulle stringhe, le _regular expressions_ (il codice è un po' involuto, non ci soffermiamo)

In [22]:
import re

incidenti[['ora', 'minuto']] = (
    incidenti['oraEvento']
    .apply(
        lambda x: ':'.join(filter(
            lambda y: all(yi.isdigit() for yi in y), 
            re.findall(r'[A-Za-z]+|\d+', x)
        ))
    ).str.split(':', expand=True)
    .astype(int)
)

Selezioniamo ora solo le colonne preprocessate 

In [23]:
incidenti = incidenti[['idUtente', 'idVeicolo', 'anno', 'mese', 'giorno', 'ora', 'minuto']]

### Target feature
Siamo finalmente pronti a creare la target feature nel dataset degli incidenti, e arricchire così quello dei viaggi, tramite merge



In [24]:
viaggi.head(2)

,idUtente,idVeicolo,carica_batteria_inizio,carica_batteria_fine,distanza_percorsa,tempo_viaggio,due_caschi_disponibili,anno,mese,giorno,ora,minuto,lat_ritiro,lon_ritiro,lat_consegna,lon_consegna,punteggioViaggio
0,u5954,v33,1.0,0.902,4.5046,800.0,1,2018,10,1,1,22,44.395187,8.943839,44.433997,8.958993,7.55
2,u3403,v103,1.0,0.826,9.2642,1661.0,1,2018,10,1,3,23,44.500364,8.903750,44.418102,8.921511,8.02


In [25]:
incidenti.head(2)

,idUtente,idVeicolo,anno,mese,giorno,ora,minuto
0,u1721,v12,2020,6,5,18,45
1,u5706,v23,2019,9,24,18,52


In [26]:
incidenti['incidente'] = 1

In [27]:
dataset = viaggi.merge(
    incidenti,
    how='left',
    on=['idUtente', 'idVeicolo', 'anno', 'mese', 'giorno', 'ora', 'minuto']
    )
dataset['incidente'].fillna(0.0, inplace=True)
dataset['incidente'] = dataset['incidente'].astype(int)

# ora che li abbiamo utilizzati per i vari merge, castiamo anche ad int gli id utenti e veicoli
dataset[['idUtente', 'idVeicolo']] = dataset[['idUtente', 'idVeicolo']].applymap(lambda x: int(x[1:]))

### Domanda

Siamo riusciti a "ritrovare" tutti gli eventi incidente nel dataset dei viaggi?

Controlliamo che il dataset sia finalmente pronto e selezioniamo solo le colonne interessanti.

⚠️ per semplicità supponiamo che le variabili temporali non influiscano sulla classificazione, e le ignoriamo da qui in poi: in generale, _non_ è detto sia una buona idea.

In [28]:
dataset.dtypes

idUtente                    int64
idVeicolo                   int64
carica_batteria_inizio    float64
carica_batteria_fine      float64
distanza_percorsa         float64
tempo_viaggio             float64
due_caschi_disponibili      int64
anno                        int64
mese                        int64
giorno                      int64
ora                         int64
minuto                      int64
lat_ritiro                float64
lon_ritiro                float64
lat_consegna              float64
lon_consegna              float64
punteggioViaggio          float64
incidente                   int32
dtype: object

In [29]:
dataset = dataset[[
    'idUtente', 'idVeicolo', 'carica_batteria_inizio',
    'carica_batteria_fine', 'distanza_percorsa', 'tempo_viaggio',
    'due_caschi_disponibili', 'lat_ritiro', 'lon_ritiro', 
    'lat_consegna', 'lon_consegna', 'punteggioViaggio', 'incidente'
]]

Salviamo il dataset

In [30]:
dataset.to_csv('./dataset_classificazione.csv', index=False)